In [ ]:
import os
from glob import glob
import math
import pandas as pd
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show

import numpy as np

from shapely.geometry import mapping
import rioxarray

import xarray
import re

import netCDF4




deaths = gpd.read_file("../migrant_deaths_analysis/death_files/ogis_migrant_deaths_091021_with_yuma.geojson")
deaths=deaths[deaths['Latitude'].notnull()]
deaths['Latitude']= deaths.Latitude.astype(float)
deaths['Longitude']= deaths.Longitude.astype(float)
deaths = gpd.GeoDataFrame(deaths, geometry=gpd.points_from_xy(deaths.Longitude,deaths.Latitude))


deaths['Reporting Date']= pd.to_datetime(deaths['Reporting Date'])
deaths['Reporting_year']= pd.to_datetime(deaths['Reporting Date']).dt.year



# download daily gidded temp data from here https://psl.noaa.gov/data/gridded/data.cpc.globaltemp.html

PATH_MAX = '/Volumes/Alex Devoid/CPC_Global_Daily_Temperature/tmax'
PATH_MIN = '/Volumes/Alex Devoid/CPC_Global_Daily_Temperature/tmin'
PATH = '/Volumes/Alex Devoid/CPC_Global_Daily_Temperature'
EXT = "*.nc"

# print(len(all_nc_files))
# print(all_nc_files)



  
def Filter(PATH1, substr,substr1 ):
    all_nc_files = [file
                 for path, subdir, files in os.walk(PATH1)
                 for file in glob(os.path.join(path, EXT))]
    return [str for str in all_nc_files if
             any(sub in str for sub in substr) and any(sub not in str for sub in substr1)]


tmax_sub = ['tmax']
tmin_sub = ['tmin']
day = ['day']
all_nc_files_tmax = Filter(PATH_MAX, tmax_sub,day)
all_nc_files_tmin = Filter(PATH_MIN, tmin_sub,day)
print(len(all_nc_files_tmin))
print(all_nc_files_tmin)
print(len(all_nc_files_tmax))
print(all_nc_files_tmax)
all_nc_files_tmin


paths_df_max = pd.DataFrame({'paths':all_nc_files_tmax})

paths_df_max['year']= paths_df_max['paths'].str.split(pat=".").str[1]


paths_df_max['year'] = pd.to_datetime(paths_df_max['year']).dt.year


paths_df_min = pd.DataFrame({'paths':all_nc_files_tmin})

paths_df_min['year']= paths_df_min['paths'].str.split(pat=".").str[1]


paths_df_min['year'] = pd.to_datetime(paths_df_min['year']).dt.year



In [ ]:
#include pall to year's file in data frame



def get_path_max(row,paths):

    
    if math.isnan(row['Reporting_year']):
        return ''
    else:
        path = paths[paths['year']==row['Reporting_year']].reset_index() 

        return path['paths'][0]
    
def get_path_max_year(row,paths):

    
    if math.isnan(row['Reporting_year']):
        return ''
    else:
        path = paths[paths['year']==row['Reporting_year']].reset_index() 

        return path['year'][0]    
    

    
    
deaths['temp_path_max'] =  deaths.apply(lambda row : get_path_max(row,paths_df_max),axis=1)
deaths['temp_path_max_year'] =  deaths.apply(lambda row : get_path_max_year(row,paths_df_max),axis=1)

deaths['temp_path_min'] =  deaths.apply(lambda row : get_path_max(row,paths_df_min),axis=1)
deaths['temp_path_min_year'] =  deaths.apply(lambda row : get_path_max_year(row,paths_df_min),axis=1)

deaths.head(1)

In [ ]:



# for path in paths:
def get_max_temp(row):
    print(row['index'])
    
    if math.isnan(row['Reporting_year']):
        return np.nan
    else:
        df = xarray.open_dataarray(row['temp_path_max'])

        df.coords['lon'] = (df.coords['lon'] + 180 + 360) % 360 - 180
        df = df.sortby(df.lon)


        df= df.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
        df.rio.write_crs("epsg:4326", inplace=True)

        df = df.sel(time=row['Reporting Date'])

        point = row['geometry']
        x,y=point.coords.xy
        value = df.sel(lon=[x[0]], lat=[y[0]], method="nearest").values[0][0]
        return value
    
    
def get_min_temp(row):
#     print(row['index'])b
    
    if math.isnan(row['Reporting_year']):
        return np.nan
    else:
        df = xarray.open_dataarray(row['temp_path_min'])

        df.coords['lon'] = (df.coords['lon'] + 180 + 360) % 360 - 180
        df = df.sortby(df.lon)


        df= df.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
        df.rio.write_crs("epsg:4326", inplace=True)

        df = df.sel(time=row['Reporting Date'])

        point = row['geometry']
        x,y=point.coords.xy
        value = df.sel(lon=[x[0]], lat=[y[0]], method="nearest").values[0][0]
        return value    
        

    
deaths = deaths.reset_index()


print('max_temp')
deaths['max_temp'] =  deaths.apply(lambda row : get_max_temp(row),axis=1)

print('min_temp')
deaths['min_temp'] =  deaths.apply(lambda row : get_min_temp(row),axis=1)
deaths.head(1)





In [ ]:
deaths.to_csv('../migrant_deaths_analysis/min_max_temp/deaths_min_max_temp.csv')